In [ ]:
import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt, mpld3
import seaborn as sns
from vqa.lib.logger import Experiment
import json
import random
#%matplotlib notebook
#mpld3.enable_notebook()

In [ ]:
def plot_line(data_x, data_y, xlabel="epochs", ylabel="accuracy %", title="Name"):
    fig, ax = plt.subplots()
    ax.set_xlabel("epochs")
    ax.set_ylabel("accuracy")
    ax.set_title(title)

    lines = ax.plot(data_y, data_x, '--', marker="o", label="lol")

    mpld3.plugins.connect(fig, mpld3.plugins.LineLabelTooltip(lines[0], label=xp_name))
    mpld3.plugins.connect(fig, mpld3.plugins.PointLabelTooltip(lines[0]))#data_y)
    mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
    #ax.grid(color='lightgrey', linestyle='solid')
    mpld3.show()
    
def plot_acc(train_accs, val_accs, fig=None, \
             xlabel="epochs", ylabel="accuracy %", title="Name"):
    if fig is None:
        fig = plt.figure()
        n = 0
    else:
        n = len(fig.axes)
        for i in range(n):
            fig.axes[i].change_geometry(n+1, 1, i+1)
    
    ax = fig.add_subplot(n+1, 1, n+1)
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    data_x = range(1, len(train_accs)+1)
    ax.plot(data_x, train_accs, '--', marker="o", label="train")
    ax.plot(data_x, val_accs, '--', marker="o", label="val")

    line_labels = ['train', 'val']
    point_labels = [train_accs, val_accs]
    for i, line in enumerate(ax.lines):
        mpld3.plugins.connect(fig, \
            mpld3.plugins.LineLabelTooltip(line, label=line_labels[i]))
        mpld3.plugins.connect(fig, \
            mpld3.plugins.PointLabelTooltip(line, labels=point_labels[i]))
    
    mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
    #ax.grid(color='lightgrey', linestyle='solid')
    #mpld3.show(open_browser=False)
    return fig
    
def plot_accs(train_accs=[], val_accs=[], names=[], fig=None, \
             xlabel="epochs", ylabel="accuracy %", title="Name"):
    if fig is None:
        fig = plt.figure()
        n = 0
    else:
        n = len(fig.axes)
        for i in range(n):
            fig.axes[i].change_geometry(n+1, 1, i+1)
    
    ax = fig.add_subplot(n+1, 1, n+1)
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

    max_length = max([len(acc) for acc in val_accs])
    data_x = range(1, max_length+1)
    for i in range(len(val_accs)):
        color = (random.random(), random.random(), random.random())
        data_x_train = range(1, len(train_accs[i])+1)
        data_x_val = range(1, len(val_accs[i])+1)
        ax.plot(data_x_train, train_accs[i],
                '--', marker="o", color=color,
                label=names[i]+"train")
        ax.plot(data_x_val, val_accs[i],
                '-', marker="o", color=color,
                label=names[i]+"val")

    point_labels = {
        'train': train_accs,
        'val': val_accs
    }
    for line_id in range(len(ax.lines)):
        label = 'train' if line_id%2==0 else 'val'
        #mpld3.plugins.connect(fig, \
        #        mpld3.plugins.LineLabelTooltip(ax.lines[line_id], label=label))
        mpld3.plugins.connect(fig, \
                mpld3.plugins.PointLabelTooltip(ax.lines[line_id], labels=point_labels[label][int(i/2)]))
    
    ax.legend()
    labels = []
    for name in names:
        labels.append(name + '_train')
        labels.append(name + '_val')
    mpld3.plugins.connect(fig, mpld3.plugins.InteractiveLegendPlugin(ax.lines, labels))
    mpld3.plugins.connect(fig, mpld3.plugins.MousePosition(fontsize=14))
    #ax.grid(color='lightgrey', linestyle='solid')
    #mpld3.show(open_browser=False)
    return fig    

def load_accs_oe(path_logger):
    dir_xp = os.path.dirname(path_logger)
    epochs = []
    for name in os.listdir(dir_xp):
        if name.startswith('epoch'):
            epochs.append(name)
    epochs = sorted(epochs, key=lambda x: float(x.split('_')[1]))
    accs = {}
    for i, epoch in enumerate(epochs):
        epoch_id = i+1
        path_acc = os.path.join(dir_xp, epoch, 'OpenEnded_mscoco_val2014_model_accuracy.json')
        if os.path.exists(path_acc):
            with open(path_acc, 'r') as f:
                data = json.load(f)
                accs[epoch_id] = data['overall']
    return accs

def sort(dict_):
    return [v for k,v in sorted(dict_.items(), \
                           key=lambda x: float(x[0]))]

def reduce(list_, num=15):
    tmp = []
    for i, val in enumerate(list_):
        if i < num:
            tmp.append(val)
    return tmp

In [ ]:
dir_logs = 'logs/vqa'
xps = []
for p in sorted(os.listdir(dir_logs)):
    if not p.startswith('.'): print(p)

In [ ]:
path_logger = os.path.join(dir_logs, 'mlb_att_samplingans,True', 'logger.json')
xp = Experiment.from_json(path_logger)
xp.logged['val']['acc1_oe'] = load_accs_oe(path_logger)
#print(xp.logged['val']['acc1_oe'])
print(xp.options)

In [ ]:
# Display accuracy & loss of one exp
def display_xp(path_logger):
    xp = Experiment.from_json(path_logger)
    xp.logged['val']['acc1_oe'] = load_accs_oe(path_logger)

    path_visu = os.path.join(os.path.dirname(path_logger),
                             'visu.html')

    val_acc1 = sort(xp.logged['val']['acc1_oe'])
    train_acc1 = sort(xp.logged['train']['acc1'])
    loss_val = sort(xp.logged['val']['loss'])
    loss_train = sort(xp.logged['train']['loss'])

    max_length = min([len(val_acc1), len(train_acc1)])
    val_acc1 = reduce(val_acc1, max_length)
    train_acc1= reduce(train_acc1, max_length)
    loss_val = reduce(loss_val, max_length)
    loss_train=reduce(loss_train, max_length)
    
    print(max(val_acc1))
    
    fig = plt.figure(figsize=(10,13))
    fig = plot_acc(train_acc1, val_acc1, fig=fig, \
                  title='max: '+str(max(val_acc1)))
    fig = plot_acc(loss_train, loss_val, fig=fig, \
                   ylabel='loss', title='min: '+str(min(loss_val)))
    fig.tight_layout()
    with open(path_visu, 'w') as f:
        mpld3.save_html(fig, f)
        
#for name in ['blocmutan']:#['seq2vec/bgru', 'mutan_noatt_2', 'mlb_att_2']:
for name in os.listdir(dir_logs):
    if not name.startswith('.'):
        path_logger = os.path.join(dir_logs, name, 'logger.json')
        print(name)
        display_xp(path_logger)

In [ ]:
# Display accuracy & loss of multiple exp
accs_train = []
accs_val = []
names = []
acc_max = ('xp name', -1)
fig = plt.figure(figsize=(20,13))
#for name in os.listdir(dir_logs):#['bgu_normal', 'mutan_noatt', 'mutan_noatt_2', 'mlb_att_2',  'mlb_att_3']:
#for name in ['seq2vec/bgru_dropout,25%', 'mutan_noatt', 'blocmutan_noatt', 'mlb_noatt_2lstm', 'mlb_noatt_2lstm_emb']:
#for name in ['blocmutan_noatt', 'blocmutan_noatt_samplingans,True']:
dir_logs = 'logs/vqa'
exps = []
exps.append('blocmutan_att_2')
exps.append('mutan_att_2')
exps.append('mlb_att_2')

for name in exps:
    if not name.startswith('.'):
        path_logger = os.path.join(dir_logs, name, 'logger.json')
        xp = Experiment.from_json(path_logger)
        xp.logged['val']['acc1_oe'] = load_accs_oe(path_logger)
        train_acc1 = sort(xp.logged['train']['acc1'])
        val_acc1 = sort(xp.logged['val']['acc1_oe'])

        max_length = min([len(val_acc1), len(train_acc1)])
        val_acc1 = reduce(val_acc1, max_length)
        train_acc1= reduce(train_acc1, max_length)

        accs_val.append(val_acc1)
        accs_train.append(train_acc1)
        print(name, max(val_acc1))
        if max(val_acc1) > acc_max[1]:
            acc_max = (name, max(val_acc1))
        names.append(name)
fig = plot_accs(accs_train, accs_val, names,
                title=acc_max[0]+' '+str(acc_max[1]), fig=fig)
path_visu = dir_logs+'/test.html'
with open(path_visu, 'w') as f:
    mpld3.save_html(fig, f)

In [ ]:
# Display accuracy & loss of multiple exp
accs_train = []
accs_val = []
names = []
acc_max = ('xp name', -1)
fig = plt.figure(figsize=(20,13))
#for name in os.listdir(dir_logs):#['bgu_normal', 'mutan_noatt', 'mutan_noatt_2', 'mlb_att_2',  'mlb_att_3']:
#for name in ['seq2vec/bgru_dropout,25%', 'mutan_noatt', 'blocmutan_noatt', 'mlb_noatt_2lstm', 'mlb_noatt_2lstm_emb']:
#for name in ['blocmutan_noatt', 'blocmutan_noatt_samplingans,True']:

dir_logs = 'logs/vqa'
list_dir = []
for name in os.listdir(dir_logs):
    if not name.startswith('.') and not name.endswith('.html'):
        list_dir.append(name)
        
for name in list_dir:
    path_logger = os.path.join(dir_logs, name, 'logger.json')
    #xp = Experiment.from_json(path_logger)
    val_acc1 = load_accs_oe(path_logger)
    #train_acc1 = sort(xp.logged['train']['acc1'])
    val_acc1 = sort(val_acc1)

    max_length = len(val_acc1)
    val_acc1 = reduce(val_acc1, max_length)
    #train_acc1= reduce(train_acc1, max_length)

    accs_val.append(val_acc1)
    #accs_train.append(train_acc1)
    print(name, max(val_acc1))
    if max(val_acc1) > acc_max[1]:
        acc_max = (name, max(val_acc1))
    names.append(name)

fig = plot_accs(accs_val, accs_val, names,
                title=acc_max[0]+' '+str(acc_max[1]), fig=fig)
path_visu = dir_logs+'/test.html'
with open(path_visu, 'w') as f:
    mpld3.save_html(fig, f)

In [ ]:
# Test displaying accuracy of one exp
val_acc1 = sort(xp.logged['val']['acc1'])
train_acc1 = sort(xp.logged['train']['acc1'])
loss_val = sort(xp.logged['val']['loss'])
loss_train = sort(xp.logged['train']['loss'])

#plot_line(val_acc1, range(1,len(val_acc1)+1))
for i in range(1,40):
    val_acc1_tmp = reduce(val_acc1, i)
    train_acc1_tmp = reduce(train_acc1, i)
    loss_val_tmp = reduce(loss_val, i)
    loss_train_tmp = reduce(loss_train, i)
    fig = plt.figure(figsize=(10,13))
    fig = plot_acc(train_acc1_tmp, val_acc1_tmp, fig=fig, \
                  title='max: '+str(max(val_acc1_tmp)))
    fig = plot_acc(loss_train_tmp, loss_val_tmp, fig=fig, \
                   ylabel='loss', title='min: '+str(min(loss_val_tmp)))
    fig.tight_layout()
    with open('test.html', 'w') as f:
        mpld3.save_html(fig, f)
    time.sleep(3)
    